In [56]:
import pandas as pd
import requests
import xmltodict
import json
from dataclasses import dataclass, asdict

In [57]:
@dataclass
class NumberSearch:
    number: int = ''
    version: str = '2.1'
    enumeration_type: any = 'NPI-2'
def get_results(search):
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    parameters = asdict(search)
    response =  requests.get(url, params=parameters)
    results = response.json()
    return results
def get_npi_info(results):
    result = results['results'][0]
    npi = result['number']
    for address in result['addresses']:
        if address['address_purpose']=='LOCATION':
            location_address_1 = address['address_1']
            location_address_2 = address['address_2']
    new_dict = {
                'NPI': npi,
                'LOCATION_ADDRESS_1': location_address_1,
                'LOCATION_ADDRESS_2': location_address_2,
            }
    return new_dict

In [58]:
filet = pd.read_excel("../../Data/Credentialling/Philadelphia_CBSA_Customer_Orders_Mar2020-Aug2020_Sample_20-10-15.xlsx")

In [59]:
npis = list(filet.drop_duplicates(['NPI_ORGANIZATION']).NPI_ORGANIZATION)

In [60]:
index = 0
dict_list = []
for npi in npis:
    results = get_results(NumberSearch(npi))
    dict_list.append(get_npi_info(results))
    print(f'{index} is done')
    index+= 1

0 is done
1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
11 is done
12 is done
13 is done
14 is done
15 is done
16 is done
17 is done
18 is done
19 is done
20 is done
21 is done
22 is done
23 is done
24 is done
25 is done
26 is done
27 is done
28 is done
29 is done
30 is done
31 is done
32 is done
33 is done
34 is done
35 is done
36 is done
37 is done
38 is done
39 is done
40 is done
41 is done
42 is done
43 is done
44 is done
45 is done
46 is done
47 is done
48 is done
49 is done
50 is done
51 is done
52 is done
53 is done
54 is done
55 is done
56 is done
57 is done
58 is done
59 is done
60 is done
61 is done
62 is done
63 is done
64 is done
65 is done
66 is done
67 is done
68 is done
69 is done
70 is done
71 is done
72 is done
73 is done
74 is done
75 is done
76 is done
77 is done
78 is done
79 is done
80 is done
81 is done
82 is done
83 is done
84 is done
85 is done


In [61]:
for thing in dict_list:
    if ' STE ' in thing['LOCATION_ADDRESS_1']:
        suite = thing['LOCATION_ADDRESS_1'].split('STE ')[1]
    elif 'SUITE' in thing['LOCATION_ADDRESS_1']:
        suite = thing['LOCATION_ADDRESS_1'].split('SUITE ')[1]
    elif ' STE ' in thing['LOCATION_ADDRESS_2']:
        suite = thing['LOCATION_ADDRESS_2'].split('STE ')[1]
    elif 'SUITE' in thing['LOCATION_ADDRESS_2']:
        suite = thing['LOCATION_ADDRESS_2'].split('SUITE ')[1]
    else:
        suite = 'None'
    thing['SUITE'] = suite

In [62]:
dict_list

[{'NPI': 1780686931,
  'LOCATION_ADDRESS_1': '1201 LANGHORNE-NEWTOWN RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1386746592,
  'LOCATION_ADDRESS_1': '18 E LAUREL RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1609417872,
  'LOCATION_ADDRESS_1': '4900 FRANKFORD AVE',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1508971441,
  'LOCATION_ADDRESS_1': '130 S BRYN MAWR AVE',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1811992084,
  'LOCATION_ADDRESS_1': '1200 OLD YORK RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1649401142,
  'LOCATION_ADDRESS_1': '3400 SPRUCE ST',
  'LOCATION_ADDRESS_2': '1-SILVERSTEIN',
  'SUITE': 'None'},
 {'NPI': 1265664544,
  'LOCATION_ADDRESS_1': '454 ENTERPRISE DR',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1831139955,
  'LOCATION_ADDRESS_1': '2701 BLAIR MILL RD',
  'LOCATION_ADDRESS_2': 'SUITE 35',
  'SUITE': '35'},
 {'NPI': 1942554308,
  'LOCATION_ADDRESS_1': '3900 WOODLAND AVE'

In [63]:
NEW = pd.DataFrame(dict_list).rename(columns={'NPI':'NPI_ORGANIZATION'})

In [70]:
datas.to_excel("../../Data/Credentialling/Philadelphia_CBSA_Customer_Orders_Mar2020-Aug2020_Sample_20-10-15.xlsx", index=False)

In [64]:
datas = pd.merge(filet, NEW, on='NPI_ORGANIZATION')

In [69]:
datas = datas[['ORDER_DATE','PRODUCT_DESC', 'CUSTOMER_NAME', 'CUSTOMER_CATEGORY_DESC', 'LOCATION_ADDRESS_1',
       'LOCATION_ADDRESS_2', 'SUITE', 'LOCATION_ZIP', 'LOCATION_CITY',
       'LOCATION_STATE','NPI_ORGANIZATION',
     'ME', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME', 'NPI_PHYSICIAN']]

In [44]:
datas.LOCATION_ZIP[80:]

80      190073101
81      190073101
82      190073101
83      190073101
84      190073101
          ...    
1995    194013353
1996    193412656
1997     80661367
1998     80661367
1999    191404160
Name: LOCATION_ZIP, Length: 1920, dtype: int64

In [65]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
datas['ME']=fix_me(list(datas['ME']))

In [66]:
def fix_zip(zip_list):
    nums = []
    for num in zip_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 8:
            num = '0' + num
        elif len(num) == 7:
            num = '00' + num
        elif len(num) == 6:
            num = '000' + num
        elif len(num) == 4:
            num = '0' + num
        nums.append(num)
    return nums
datas['LOCATION_ZIP']=fix_zip(list(datas['LOCATION_ZIP']))

In [55]:
datas.to_excel("../../Data/Credentialling/Philadelphia_CBSA_Customer_Orders_Mar2020-Aug2020_Sample.xlsx", index=False)